In [2]:
from langchain_core.tools import tool
@tool
def multiply(a:int,b:int)->int:
    """Multiply two numbers."""
    return a*b
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


##### This @tool decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. This @tool decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. 

In [3]:
# as a param to tool (input schema), one can also add annotations, nested schems and  other features too
from typing import Annotated,List
@tool
def multiply_by_max(
    a: Annotated[str, "scale factor"],
    b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)

In [4]:
print(multiply_by_max.name)

multiply_by_max


In [5]:
print(multiply_by_max.description)
# print(multiply.args)

Multiply a by the maximum of b.


In [6]:
print(multiply_by_max.args)

{'a': {'title': 'A', 'type': 'string'}, 'b': {'title': 'B', 'type': 'array', 'items': {'type': 'integer'}}}


In [7]:
# as such aap input m ek json schema ya [pydantic ] bhi de skte ho
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

# by default tool ka naam whi hota 
# jo function ka naam hai jiske upar @ decorator lga hai
# otherwise khud se spcify name like here I did 
@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiplication-tool
Multiply two numbers.
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}
True


In [8]:
from langchain_core.tools import StructuredTool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 5}))


6
10


In [9]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()


True

In [12]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper()
tool=WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({"query":"langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.



Page: DataStax


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [15]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest",temperature=0.2)

In [44]:
llm=ChatGroq(model="llama-3.1-8b-instant",temperature=0.2)

In [16]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent
tool=load_tools(["wikipedia"],llm=llm)

In [17]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [20]:
agent.run("what is llama and who create this llm model?")



> Entering new AgentExecutor chain...
Thought: I need to find information about "llama" as a large language model (LLM).  Wikipedia should be a good resource for this.

Action: wikipedia
Action Input: LLaMA (large language model)

Observation: Page: Llama (language model)
Summary: Llama (Large Language Model Meta AI, formerly stylized as LLaMA) is a family of autoregressive large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 3.2, released in September 2024.
Model weights for the first version of Llama were made available to the research community under a non-commercial license, and access was granted on a case-by-case basis. Unauthorized copies of the model were shared via BitTorrent. In response, Meta AI issued DMCA takedown requests against repositories sharing the link on GitHub. Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted some commercial use. Llama models are trai

'LLaMA (Large Language Model Meta AI) is a family of large language models (LLMs) created by Meta AI.'

In [21]:
# the above thing works good,
# as the mentioned says it will search inside wikipedia for this informati

In [37]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [38]:
search = TavilySearchResults()

In [39]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1731848824, 'localtime': '2024-11-17 05:07'}, 'current': {'last_updated_epoch': 1731848400, 'last_updated': '2024-11-17 05:00', 'temp_c': 8.3, 'temp_f': 46.9, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 3.1, 'wind_kph': 5.0, 'wind_degree': 124, 'wind_dir': 'SE', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 0, 'feelslike_c': 7.8, 'feelslike_f': 46.1, 'windchill_c': 12.0, 'windchill_f': 53.6, 'heatindex_c': 11.8, 'heatindex_f': 53.2, 'dewpoint_c': 8.4, 'dewpoint_f': 47.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 5.0, 'gust_kph': 8.0}}"},
 {'url': 'https://www.timeanddate.com/weather/usa/san-

In [40]:
tools=[search]


In [27]:
from langchain.agents import create_tool_calling_agent
from langchain import hub

In [29]:
prompt=hub.pull("hwchase17/openai-functions-agent")

c:\Users\Charanjeet Juneja\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [31]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='

In [32]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

agent_scratchpad: The agent does not call a tool just once to get the desired answer, but has a structure that calls tools repeatedly until the desired answer is obtained. Each time you call a tool, what the previous call was like, information about the previous call, and the result are stored in this field.

In [45]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [46]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [49]:
agent_executor.invoke({"input": "whats are the multiple mail addresses through which I can contact Zomato for purposes like legal, support, and interview?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Zomato contact email addresses legal support interview'}`


[{'url': 'https://www.moneycontrol.com/company-facts/zomato/management/Z', 'content': 'Get Zomato contact details, Registered Address of Zomato along with Zomato management information. ... Interview Series Business In The Week Ahead ... Email: companysecretary@zomato.com'}, {'url': 'https://www.grievanceofficer.com/grievance-officer-zomato/', 'content': 'Grievance Officer at Zomato - Grievance Officer Grievance Officer Grievance Officers You are here: Home / Grievance Officer at Zomato Grievance Officer at Zomato Grievance Redressal Officer Email address: legal@zomato.com Grievance Redressal Officer Email address: legal@zomato.com my order value is ₹700 But zomato team says they cant do anyting and they said they can only waive of ₹30 later ₹300, here we are suffering here food is nt warm either the restaurents are not charged pwn

{'input': 'whats are the multiple mail addresses through which I can contact Zomato for purposes like legal, support, and interview?',
 'output': "Based on the search results, it appears that Zomato's contact email addresses for purposes like legal, support, and interview are:\n\n* companysecretary@zomato.com\n* legal@zomato.com\n\nPlease note that these email addresses may be subject to change, and it's always best to check Zomato's official website or contact their customer support for the most up-to-date information."}

In [36]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...
I am sorry, I cannot fulfill this request. The available tools lack the desired functionality.

> Finished chain.


{'input': 'whats the weather in sf?',
 'output': 'I am sorry, I cannot fulfill this request. The available tools lack the desired functionality.'}